# Sentimental Analysis of twitter data using deep learning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Importing Datasets**

In [ ]:
train = pd.read_csv('dataset/train.csv', encoding='latin-1')
test = pd.read_csv('dataset/test.csv', encoding='latin-1')

In [ ]:
train.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [ ]:
train.shape

(99989, 3)

In [ ]:
test.shape

(299989, 2)

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
sentiment = train["SentimentText"][0]
sentiment = str(re.sub('[^a-zA-Z]', ' ', sentiment)).lower().strip()

In [ ]:
sentiment

'is so sad for my apl friend'

In [ ]:
ps = PorterStemmer()
sentiment = [ps.stem(word) for word in sentiment if not word in set(stopwords.words('english'))]

In [ ]:
sentiment

[' ', ' ', ' ', 'f', 'r', ' ', ' ', 'p', 'l', ' ', 'f', 'r', 'e', 'n']

In [ ]:
data = []
for i in range(train.shape[0]):
  sentiment = train["SentimentText"][i]
  sentiment = str(re.sub('[^a-zA-Z]', ' ', sentiment)).lower().strip().split()
  sentiment = [ps.stem(word) for word in sentiment if not word in set(stopwords.words('english'))]
  sentiment = ' '.join(sentiment)
  data.append(sentiment)


In [ ]:
data[:20]

['sad apl friend',
 'miss new moon trailer',
 'omg alreadi',
 'omgaga im sooo im gunna cri dentist sinc supos get crown put min',
 'think mi bf cheat',
 'worri much',
 'juuuuuuuuuuuuuuuuussssst chillin',
 'sunni work tomorrow tv tonight',
 'hand uniform today miss alreadi',
 'hmmmm wonder number',
 'must think posit',
 'thank hater face day',
 'weekend suck far',
 'jb isnt show australia',
 'ok that win',
 'lt way feel right',
 'awhh man complet useless rt funni twitter http myloc hx',
 'feel strang fine gonna go listen semison celebr',
 'huge roll thunder scari',
 'cut beard grow well year gonna start shaunamanu happi meantim']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1000, stop_words='english')
cv.fit(data)
X = cv.transform(data).toarray()

In [ ]:
X.shape

(99989, 1000)

In [ ]:
cv.vocabulary_

{'sad': 727,
 'friend': 354,
 'miss': 571,
 'new': 597,
 'moon': 580,
 'omg': 620,
 'alreadi': 29,
 'im': 450,
 'sooo': 799,
 'cri': 227,
 'sinc': 776,
 'min': 568,
 'think': 862,
 'worri': 970,
 'sunni': 838,
 'work': 968,
 'tomorrow': 877,
 'tv': 899,
 'tonight': 878,
 'hand': 398,
 'today': 874,
 'wonder': 965,
 'number': 609,
 'posit': 666,
 'thank': 859,
 'face': 307,
 'day': 243,
 'weekend': 951,
 'suck': 833,
 'far': 316,
 'isnt': 466,
 'australia': 74,
 'ok': 617,
 'win': 958,
 'lt': 533,
 'way': 943,
 'feel': 325,
 'right': 717,
 'man': 545,
 'complet': 201,
 'funni': 357,
 'twitter': 905,
 'http': 435,
 'myloc': 588,
 'fine': 333,
 'gonna': 373,
 'listen': 516,
 'celebr': 168,
 'huge': 438,
 'roll': 721,
 'scari': 737,
 'cut': 231,
 'grow': 385,
 'year': 990,
 'start': 817,
 'happi': 401,
 'caus': 166,
 'els': 285,
 'follow': 343,
 'pretti': 675,
 'awesom': 80,
 'write': 975,
 'blog': 119,
 'tweet': 900,
 'myspac': 589,
 'shut': 768,
 'lost': 529,
 'head': 406,
 'pull': 686,


In [ ]:
y = train.iloc[:, 1].values

In [ ]:
y.shape

(99989,)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 7)

In [ ]:
print("X_train : ", X_train.shape, "\nX_test : ", X_test.shape, "\ny_train : ", y_train.shape, "\ny_test : ", y_test.shape)

X_train :  (79991, 1000) 
X_test :  (19998, 1000) 
y_train :  (79991,) 
y_test :  (19998,)


**Model Building**


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(output_dim = 128, init = "uniform", activation = 'relu', input_dim = 1000))
model.add(Dense(output_dim = 64, init = 'uniform', activation = 'relu'))
model.add(Dense(output_dim = 64, init = 'uniform', activation = 'relu'))
model.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64, kernel_initializer="uniform")`
  


In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 100, batch_size = 100)

Epoch 1/100
79991/79991 [==============================] - 4s 53us/step - loss: 0.5569 - accuracy: 0.7060
Epoch 2/100
79991/79991 [==============================] - 4s 46us/step - loss: 0.5185 - accuracy: 0.7375
Epoch 3/100
79991/79991 [==============================] - 4s 46us/step - loss: 0.4949 - accuracy: 0.7539
Epoch 4/100
79991/79991 [==============================] - 4s 46us/step - loss: 0.4517 - accuracy: 0.7837
Epoch 5/100
79991/79991 [==============================] - 4s 46us/step - loss: 0.3940 - accuracy: 0.8167
Epoch 6/100
79991/79991 [==============================] - 4s 46us/step - loss: 0.3347 - accuracy: 0.8457
Epoch 7/100
79991/79991 [==============================] - 4s 45us/step - loss: 0.2864 - accuracy: 0.8675
Epoch 8/100
79991/79991 [==============================] - 4s 46us/step - loss: 0.2496 - accuracy: 0.8840
Epoch 9/100
79991/79991 [==============================] - 4s 46us/step - loss: 0.2222 - accuracy: 0.8953
Epoch 10/100
79991/79991 [====================

# Saving the model

In [ ]:
import pickle
pickle.dump(cv, open('cv.pkl', 'wb'))

In [ ]:
model.save('model.h5')

# Making predictions

In [ ]:
predictions = [ 1 if x > 0.5 else 0 for x in model.predict(X_test)]

In [ ]:
predictions[:20]

[1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(confusion_matrix(y_test, predictions))

[[5159 3653]
 [2645 8541]]


In [ ]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.66      0.59      0.62      8812
           1       0.70      0.76      0.73     11186

    accuracy                           0.69     19998
   macro avg       0.68      0.67      0.68     19998
weighted avg       0.68      0.69      0.68     19998



**End**
